In [ ]:
# default_exp core

In [ ]:
# export
import numpy as np
import json
from pathlib import Path
from nbdev.imports import test_eq

In [ ]:
# hide
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script

# Core
> Functions that implement basic functionality that will be used in the library.

## Util functions

A set of functions that provide usefull functionality

In [ ]:
# export
def filter_files(files, include=[], exclude=[]):
    "Filter list of files using a list of strings to inculde and/or exclude"
    for incl in include:
        files = [f for f in files if incl in f.name]
    for excl in exclude:
        files = [f for f in files if excl not in f.name]
    return sorted(files)

def ls(x, recursive=False, include=[], exclude=[]):
    "List files in folder, if recursive is True also list subfolders"
    if not recursive:
        out = list(x.iterdir())
    else:
        out = [o for o in x.glob('**/*')]
    out = filter_files(out, include=include, exclude=exclude)
    return sorted(out)

Path.ls = ls

def hdf_attr_check(attr, hdf, default):
    "Check if attribute is in hdf_attr_dict and return default"
    return default if not hasattr(hdf, attr) else hdf.__getattr__(attr)

def dict2json(data:dict, file):
    "Writes json file from dict"
    with open(file, 'w') as f:
        f.write(json.dumps(data))

Examples:

In [ ]:
path = Path('.')
path.ls()

[Path('.gitattributes'),
 Path('.ipynb_checkpoints'),
 Path('.last_checked'),
 Path('00_core.ipynb'),
 Path('01_geo.ipynb'),
 Path('02_data.ipynb'),
 Path('03_models.ipynb'),
 Path('04_predict.ipynb'),
 Path('04b_nrt.ipynb'),
 Path('04c_historical.ipynb'),
 Path('05_train.ipynb'),
 Path('06_cli.ipynb'),
 Path('07_web.ipynb'),
 Path('data'),
 Path('hide'),
 Path('historicalPT'),
 Path('images'),
 Path('index.ipynb'),
 Path('tutorial.australia2020.ipynb'),
 Path('tutorial.australia2020_100m.ipynb')]

In [ ]:
path = Path('.')
path.ls(include=['.ipynb'])

[Path('.ipynb_checkpoints'),
 Path('00_core.ipynb'),
 Path('01_geo.ipynb'),
 Path('02_data.ipynb'),
 Path('03_models.ipynb'),
 Path('04_predict.ipynb'),
 Path('04b_nrt.ipynb'),
 Path('04c_historical.ipynb'),
 Path('05_train.ipynb'),
 Path('06_cli.ipynb'),
 Path('07_web.ipynb'),
 Path('index.ipynb'),
 Path('tutorial.australia2020.ipynb'),
 Path('tutorial.australia2020_100m.ipynb')]

In [ ]:
path = Path('.')
path.ls(include=['.ipynb'], exclude=['_checkpoints'])

[Path('00_core.ipynb'),
 Path('01_geo.ipynb'),
 Path('02_data.ipynb'),
 Path('03_models.ipynb'),
 Path('04_predict.ipynb'),
 Path('04b_nrt.ipynb'),
 Path('04c_historical.ipynb'),
 Path('05_train.ipynb'),
 Path('06_cli.ipynb'),
 Path('07_web.ipynb'),
 Path('index.ipynb'),
 Path('tutorial.australia2020.ipynb'),
 Path('tutorial.australia2020_100m.ipynb')]

In [ ]:
# export        
def monthlen(year, month):
    "Gives lenght of the month"
    base = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    if (year % 4) == 0:
        if (year % 100) == 0:
            if (year % 400) == 0:
                base[1] += 1
        else:
            base[1] += 1
    return base[month-1]

In [ ]:
year = 2000
month = 2
monthlen(year, month)

29

In [ ]:
# export
class InOutPath():
    """Keeps track of an input and a output path. Creates paths if they don't exist and mkdir=True"""
    def __init__(self, input_path:str, output_path:str, mkdir=True):
        if isinstance(input_path, str): input_path = Path(input_path)
        if isinstance(output_path, str): output_path = Path(output_path)
        self.input_path = input_path
        self.output_path = output_path
        if mkdir: self.mkdirs()

    @property
    def src(self):
        "Shortcut to input_path"
        return self.input_path

    @property
    def dst(self):
        "Shortcut to output_path"
        return self.output_path

    def mkdirs(self):
        self.input_path.mkdir(exist_ok=True, parents=True)
        self.output_path.mkdir(exist_ok=True, parents=True)

    def __truediv__(self, s):
        return InOutPath(self.src/s, self.dst/s)

    def __repr__(self):
        return '\n'.join([f'{i}: {o}' for i, o in self.__dict__.items()]) + '\n'

In [ ]:
show_doc(InOutPath.src)
show_doc(InOutPath.dst)

<h4 id="InOutPath.src" class="doc_header"><code>InOutPath.src</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Shortcut to input_path

<h4 id="InOutPath.dst" class="doc_header"><code>InOutPath.dst</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Shortcut to output_path

In [ ]:
# export
class ProjectPath():
    def __init__(self, path:Path, config_dir='config', ladsweb_dir='ladsweb',
                 hotspots_dir='hotspots', dataset_dir='dataset', outputs_dir='outputs',
                 web_dir='web'):
        if isinstance(path, str): path = Path(path)
        self.path     = path
        self.config   = path/config_dir
        self.ladsweb  = path/ladsweb_dir
        self.hotspots = path/hotspots_dir
        self.dataset  = path/dataset_dir
        self.outputs  = path/outputs_dir
        self.web      = path/web_dir
        self.create_dirs([self.path, self.config, self.ladsweb, self.hotspots,
                          self.dataset, self.outputs, self.web])
        
    @staticmethod
    def create_dirs(paths_list):
        "Create directories in list."
        for o in paths_list: o.mkdir(exist_ok=True)

In [ ]:
show_doc(ProjectPath.create_dirs)

<h4 id="ProjectPath.create_dirs" class="doc_header"><code>ProjectPath.create_dirs</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>ProjectPath.create_dirs</code>(**`paths_list`**)

Create directories in list.

In [ ]:
paths = ProjectPath('data')
test_eq(sorted([o.name for o in paths.path.iterdir()]), 
        ['config', 'dataset', 'hotspots', 'ladsweb', 'outputs', 'web'])

In [ ]:
# hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_geo.ipynb.
Converted 02_data.ipynb.
Converted 03_models.ipynb.
Converted 04_predict.ipynb.
Converted 04b_nrt.ipynb.
Converted 04c_historical.ipynb.
Converted 05_train.ipynb.
Converted 06_cli.ipynb.
Converted 07_web.ipynb.
Converted index.ipynb.
Converted tutorial.australia2020.ipynb.
Converted tutorial.australia2020_100m.ipynb.
